# Analysis of A05 results

## Data loading

In [ ]:
from curve_analyzer.utils.data_processing import load_curves, load_trait, filter_df
curves = load_curves() # load all curves
df_a05 = load_trait("a05") # load all results for trait a05
df_a05 = df_a05.merge(curves, "left", "curve") # join curve metadata to trait results

## Data filtering

In [ ]:
import difflib
import ipywidgets as widgets
from ipywidgets import interact, interactive, HBox, VBox
from curve_analyzer.definitions import STD_SOURCES, STD_BITLENGTHS, ALL_COFACTORS

style = {'description_width': 'initial'}

def multi_checkbox_widget(preselected, others):
    """ Inspired by from https://gist.github.com/pbugnion/5bb7878ff212a0116f0f1fbc9f431a5c """
    """ Widget with a search field and lots of checkboxes """
    
    # Get rid of overlapping elements and convert everything to strings, but sort numerically when possible
    others_strings = list(map(str, set(others)-set(preselected)))
    try:
        others = sorted(others_strings, key=int)
    except ValueError:
        others = sorted(others_strings)
    preselected = list(map(str, preselected))
    
    search_widget = widgets.Text(layout=widgets.Layout(width="auto"))
    options_dict = {**{description: widgets.Checkbox(description=description, value=True) for description in preselected},
                    **{description: widgets.Checkbox(description=description, value=False) for description in others}}
    all_descriptions = preselected + others
    options = [options_dict[description] for description in all_descriptions]
    options_widget = widgets.VBox(options, layout=widgets.Layout(overflow='scroll', width="auto"))
    multi_select = widgets.VBox([search_widget, options_widget], layout=widgets.Layout(width="auto"))

    # Wire the search field to the checkboxes
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = [options_dict[description] for description in all_descriptions]
        else:
            # Filter by search field using difflib.
            close_matches = difflib.get_close_matches(search_input, all_descriptions, cutoff=0.0)
            new_options = [options_dict[description] for description in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    return multi_select

source_choice = multi_checkbox_widget(preselected=["std", "sim"], others=STD_SOURCES)
bitlength_choice = multi_checkbox_widget(preselected=[128, 160, 192, 224, 256 ], others=STD_BITLENGTHS)
cofactor_choice = multi_checkbox_widget(preselected=[1], others=ALL_COFACTORS)
l_choice = multi_checkbox_widget(preselected=[2], others=[2, 3, 5, 7])

choices = [source_choice, bitlength_choice, cofactor_choice, l_choice]
HBox(choices, layout=widgets.Layout(justify_content='space-around', height='250px'))

## Data inspection

In [ ]:
def get_results(choices):
    for choice in choices:
        yield [w.description for w in choice.children[1].children if w.value]

sources, bitlengths, cofactors, ls = get_results(choices)
df = filter_df(df_a05, bitlengths=bitlengths, sources=sources, cofactors=cofactors)
df = df[df["l"].isin(ls)]
df

In [ ]:
df.agg(set) # display only distinct values

In [ ]:
df["bitlength"]

In [ ]:
df.bitlength

In [ ]:
df.describe()

## Data visualization

In [ ]:
from curve_analyzer.utils.visualization import normalized_barplot, normalized_bubbleplot

In [ ]:
for column in ("least", "full"):
    normalized_barplot(df, column, title=f"Normalized barplot of {column}")

In [ ]:
for l in ls:
    normalized_bubbleplot(df[df["l"] == l], f"least", f"full", title=f"Normalized bubble plot for l={l}")

## Playground

In [ ]:
sorted(["ab","ac"], key=int)

In [ ]:
sorted(['mnt', 'wtls', 'nums', 'x962', 'x963', 'bn', 'nist', 'oscaa', 'other', 'bls', 'anssi', 'secg', 'gost', 'brainpool', 'oakley'])